# Word2Vec (Skipgram )

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [6]:
np.__version__, torch.__version__

('1.24.0', '1.12.1')

In [7]:
import matplotlib
matplotlib.__version__

'3.6.2'

## 1. Load data

## 2. Prepare train data
You move the window along, and create those tuples as we said in class

## 3. Model

$$J(\theta) = -\frac{1}{T}\sum_{t=1}^{T}\sum_{\substack{-m \leq j \leq m \\ j \neq 0}}\log P(w_{t+j} | w_t; \theta)$$

where $P(w_{t+j} | w_t; \theta) = $

$$P(o|c)=\frac{\exp(\mathbf{u_o^{\top}v_c})}{\sum_{w=1}^V\exp(\mathbf{u_w^{\top}v_c})}$$

where $o$ is the outside words and $c$ is the center word

## 4. Training

## 5. Plot the embeddings

Is really the related stuff are close to each other, and vice versa?

## 6. Cosine similarity

How do (from scratch) calculate cosine similarity?